In [34]:
import sympy as sp
from IPython.display import display, Math
import time
start_time = time.time()

In [35]:
# x, y, z sú súradnice bodov v R^3
x, y, z = sp.symbols('x y z')
# t, s sú parametre z intervalu I
t, s = sp.symbols('t, s')
# a, b sú konštanty pre škálovanie elipsoidu, a - dotykový smer, b - normálový a binormálový smer
a, b = sp.symbols('a b')
# všetky ostatné symboly sú parametre v parametrizácii krivky parametric_curve
c, d = sp.symbols('c d')

In [36]:
# Vstupné parametre
parametric_curve = sp.Matrix([sp.sin(t), sp.cos(t),t]) # parametrizácia krivky
# interval vyčíslenia krivky
s_min = -10
s_max = 11
# parametre škálovania
a_0 = 2
b_0 = 1
# parametre krivky
c_0 = 1
d_0 = 2

In [37]:
# Vypočíta Frenetov repér pre parametric_curve
"""vstup: parametric_curve : sympy.Matrix
    výstup: dotyčnica, normála, binormála : tuple"""

def frenet_frame(parametric_curve):
    t = sp.Symbol('t')
    parametric_curve_first_derivation = sp.diff(parametric_curve, t)
    T = parametric_curve_first_derivation / sp.sqrt(parametric_curve_first_derivation.dot(parametric_curve_first_derivation))

    N = sp.Matrix([0, -T[2], T[1]])
    N_norm = sp.sqrt(N.dot(N))
    N = N / N_norm

    B = T.cross(N)
    B_norm = sp.sqrt(B.dot(B))
    B = B / B_norm
    return T, N, B

def make_matrix(T, N, B):
    A = sp.Matrix.hstack(T, N, B)
    return A.T

T, N, B = frenet_frame(parametric_curve)
A = make_matrix(T, N, B)
display(Math(sp.latex(A)))

<IPython.core.display.Math object>

In [38]:
# Maticou A vynásobíme vektor (X-parametric_curve) a tak zmeníme bázu elipsoidu na Frentov repér
vector = sp.Matrix([x - parametric_curve[0], y - parametric_curve[1], z - parametric_curve[2]])
result = A * vector
#display(Math(sp.latex(result)))

In [39]:
ellipsoid = result[0]**2/a**2 + result[1]**2/b**2 + result[2]**2/b**2
# Pre účely vykresľovania substitúcia t za s
ellipsoid_in_frenet_basis = ellipsoid.subs(t, s)
ellipsoid_in_frenet_basis = sp.simplify(ellipsoid_in_frenet_basis)
# Hurá, máme elipsoid v inej báze, ak odčítame 1 a položíme výraz rovný 0
# Rovnica elipsoidu v báze Frenetovho repéra
#print("Rovnica systému: ")
#display(Math(sp.latex((ellipsoid_in_frenet_basis - 1))))

In [40]:
# Derivácia jednoparametrického systému elipsoidov 
derivation = sp.diff(ellipsoid_in_frenet_basis, s)
#derivation = sp.simplify(sp.diff(ellipsoid_in_frenet_basis, s))
#print("Derivácia sytému: ")
#display(Math(sp.latex(derivation)))

In [41]:
print("Systém: ")
#display(Math(sp.latex((ellipsoid_in_frenet_basis - 1))))
ellipsoid_in_frenet_basis = sp.simplify(ellipsoid_in_frenet_basis)
print(sp.latex((ellipsoid_in_frenet_basis - 1)) + '=0')

print("Derivácia sytému: ")
#display(Math(sp.latex(derivation)))
derivation = sp.simplify(derivation)
print(sp.latex(derivation) + '=0')

Systém: 
-1 + \frac{8 a^{2} \left(- y + \left(s - z\right) \sin{\left(s \right)} + \cos{\left(s \right)}\right)^{2} + a^{2} \left(2 s \cos{\left(s \right)} - x \cos{\left(2 s \right)} + 3 x + y \sin{\left(2 s \right)} - 2 z \cos{\left(s \right)} - 4 \sin{\left(s \right)}\right)^{2} + 4 b^{2} \left(\sin^{2}{\left(s \right)} + 1\right) \left(s - x \cos{\left(s \right)} + y \sin{\left(s \right)} - z\right)^{2}}{8 a^{2} b^{2} \left(\sin^{2}{\left(s \right)} + 1\right)}=0
Derivácia sytému: 
- \frac{s x \sin{\left(s \right)}}{b^{2}} - \frac{s y \cos{\left(s \right)}}{b^{2}} + \frac{s}{b^{2}} + \frac{x^{2} \sin{\left(2 s \right)}}{2 b^{2}} + \frac{x y \cos{\left(2 s \right)}}{b^{2}} + \frac{x z \sin{\left(s \right)}}{b^{2}} - \frac{x \cos{\left(s \right)}}{b^{2}} - \frac{y^{2} \sin{\left(2 s \right)}}{2 b^{2}} + \frac{y z \cos{\left(s \right)}}{b^{2}} + \frac{y \sin{\left(s \right)}}{b^{2}} - \frac{z}{b^{2}} + \frac{s x \sin{\left(s \right)}}{a^{2}} + \frac{s y \cos{\left(s \right)}}{a^{2}} +

In [42]:
# Výpočet, o aký typ plochy ide
# Najprv potrebujeme extrahovať koeficienty z derivácie
expanded_expression = sp.expand(derivation)
#print(expanded_expression)

coefficients_x_sq = expanded_expression.coeff(x**2)
coefficients_y_sq = expanded_expression.coeff(y**2)
coefficients_z_sq = expanded_expression.coeff(z**2)
coefficients_xy = expanded_expression.coeff(x*y)
coefficients_yz = expanded_expression.coeff(y*z)
coefficients_xz = expanded_expression.coeff(x*z)

# Tu bolo potrebné odstrániť všetky koeficienty pri členoch x*y, x*z,..
coef_x = expanded_expression.coeff(x)
terms_x = sp.Add(*coef_x.args) if isinstance(coef_x, sp.Add) else coef_x
terms_x_filtered = [term for term in sp.Add.make_args(terms_x) if not term.has(y) and not term.has(z)]
coefficients_x = sp.Add(*terms_x_filtered)

coef_y = expanded_expression.coeff(y)
terms_y = sp.Add(*coef_y.args) if isinstance(coef_y, sp.Add) else coef_y
terms_y_filtered = [term for term in sp.Add.make_args(terms_y) if not term.has(x) and not term.has(z)]
coefficients_y = sp.Add(*terms_y_filtered)

coef_z = expanded_expression.coeff(z)
terms_z = sp.Add(*coef_z.args) if isinstance(coef_z, sp.Add) else coef_z
terms_z_filtered = [term for term in sp.Add.make_args(terms_z) if not term.has(x) and not term.has(y)]
coefficients_z = sp.Add(*terms_z_filtered)

coef_constant = expanded_expression.coeff(1)
terms = sp.Add(*coef_constant.args) if isinstance(coef_constant, sp.Add) else coef_constant
terms_filtered = [term for term in sp.Add.make_args(terms) if not term.has(x) and not term.has(y) and not term.has(z)]
coefficients_constant = sp.Add(*terms_filtered)

# Print koeficientov
'''print("Coefficient of x^2:", coefficients_x_sq)
print("Coefficient of y^2:", coefficients_y_sq)
print("Coefficient of z^2:", coefficients_z_sq)
print("Coefficient of xy:", coefficients_xy)
print("Coefficient of yz:", coefficients_yz)
print("Coefficient of xz:", coefficients_xz)
print("Coefficient of x:", coefficients_x)
print("Coefficient of y:", coefficients_y)
print("Coefficient of z:", coefficients_z)
print("Coefficient of constant term:", coefficients_constant)'''

'print("Coefficient of x^2:", coefficients_x_sq)\nprint("Coefficient of y^2:", coefficients_y_sq)\nprint("Coefficient of z^2:", coefficients_z_sq)\nprint("Coefficient of xy:", coefficients_xy)\nprint("Coefficient of yz:", coefficients_yz)\nprint("Coefficient of xz:", coefficients_xz)\nprint("Coefficient of x:", coefficients_x)\nprint("Coefficient of y:", coefficients_y)\nprint("Coefficient of z:", coefficients_z)\nprint("Coefficient of constant term:", coefficients_constant)'

In [43]:
def matrix_of_quadrics(A, B, C, D, E, F, G, H, I, J):
    row1 = [A, D/2, E/2, G/2]
    row2 = [D/2, B, F/2, H/2]
    row3 = [E/2, F/2, C, I/2]
    row4 = [G/2, H/2, I/2, J]
    matrix = sp.Matrix([row1, row2, row3, row4])
    return matrix

A = coefficients_x_sq
B = coefficients_y_sq
C = coefficients_z_sq
D = coefficients_xy
E = coefficients_yz
F = coefficients_xz
G = coefficients_x
H = coefficients_y
I = coefficients_z
J = coefficients_constant

result_matrix = matrix_of_quadrics(A, B, C, D, E, F, G, H, I, J)
#print("Matica kvadratickej plochy: ")
#display(Math(sp.latex(result_matrix)))

In [44]:
# Typ plochy
expression = A*B-D**2/4 + B*C - F**2/4 + C*J-I**2/4
#expression = sp.simplify(expression)
#print("Výraz: ")
#display(Math(sp.latex(expression)))

In [45]:
# Typ plochy vieme jasne dourčiť pre špecifikáciu parametrov
s_values = range(s_min, s_max)
is_negative = True
for s_val in s_values:
        if expression.subs({s: s_val, a: a_0, b: b_0, c: c_0, d: d_0}) >= 0:
            is_negative = False
            break

if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")


Výraz je pre zadané hodnoty záporný. 


In [46]:
# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Elapsed time:", elapsed_time, "seconds")

Elapsed time: 31.81833815574646 seconds


In [47]:
print(parametric_curve)
print("Systém: ")
#display(Math(sp.latex((ellipsoid_in_frenet_basis - 1))))
print(sp.latex((ellipsoid_in_frenet_basis - 1)) + '=0')

print("Derivácia sytému: ")
#display(Math(sp.latex(derivation)))
print(sp.latex(derivation) + '=0')

print("Čas bez optimalizácie: ")
print(elapsed_time, "seconds")

print("Výraz: ")
display(Math(sp.latex(expression)))
if is_negative:
    print("Výraz je pre zadané hodnoty záporný. ")
else:
    print("Výraz nie je pre zadané hodnoty záporný. ")

Matrix([[sin(t)], [cos(t)], [t]])
Systém: 
-1 + \frac{8 a^{2} \left(- y + \left(s - z\right) \sin{\left(s \right)} + \cos{\left(s \right)}\right)^{2} + a^{2} \left(2 s \cos{\left(s \right)} - x \cos{\left(2 s \right)} + 3 x + y \sin{\left(2 s \right)} - 2 z \cos{\left(s \right)} - 4 \sin{\left(s \right)}\right)^{2} + 4 b^{2} \left(\sin^{2}{\left(s \right)} + 1\right) \left(s - x \cos{\left(s \right)} + y \sin{\left(s \right)} - z\right)^{2}}{8 a^{2} b^{2} \left(\sin^{2}{\left(s \right)} + 1\right)}=0
Derivácia sytému: 
- \frac{s x \sin{\left(s \right)}}{b^{2}} - \frac{s y \cos{\left(s \right)}}{b^{2}} + \frac{s}{b^{2}} + \frac{x^{2} \sin{\left(2 s \right)}}{2 b^{2}} + \frac{x y \cos{\left(2 s \right)}}{b^{2}} + \frac{x z \sin{\left(s \right)}}{b^{2}} - \frac{x \cos{\left(s \right)}}{b^{2}} - \frac{y^{2} \sin{\left(2 s \right)}}{2 b^{2}} + \frac{y z \cos{\left(s \right)}}{b^{2}} + \frac{y \sin{\left(s \right)}}{b^{2}} - \frac{z}{b^{2}} + \frac{s x \sin{\left(s \right)}}{a^{2}} + \frac{s

<IPython.core.display.Math object>

Výraz je pre zadané hodnoty záporný. 
